In [3]:
EG = "/Users/joregan/Playing/merged_annotations/hsi_4_0717_209_002.json"

In [1]:
import json

In [4]:
with open(EG) as inf:
    data = json.load(inf)

In [9]:
SAMPLE = """
{
  "2": {
    "general": {
      "id": 2,
      "start": 5.970287890159987,
      "end": 9.551266621561616
    },
    "high_level": {
      "topic_name": "reference_real",
      "current_topic": "vase",
      "topic_change": false,
      "topic_duration_id": 0,
      "spatial_reference": "fourth vase",
      "referenced_object": [
        "vase"
      ]
    },
    "low_level": {
      "resolved_references": {
        "the fourth vase": "vase",
        "it": "vase"
      },
      "spatial_relationships": [],
      "resolved_adverbs": []
    },
    "snippet": "But I'm wondering about the fourth vase. Where is it? Did you break it?"
  }
}
"""

In [10]:
thing = json.loads(SAMPLE.strip())

In [13]:
thing["2"].keys()

dict_keys(['general', 'high_level', 'low_level', 'snippet'])

In [14]:
thing["2"]["low_level"]

{'resolved_references': {'the fourth vase': 'vase', 'it': 'vase'},
 'spatial_relationships': [],
 'resolved_adverbs': []}